In [1]:
# coding: utf-8
from __future__ import print_function, division, absolute_import
import sys
import time
import numpy as np
import tensorflow as tf

In [2]:
!rm -fr logdir
!mkdir -p logdir

In [3]:
from tensorflow.examples.tutorials.mnist.input_data \
  import read_data_sets

mnist = read_data_sets('./mnist', one_hot=False)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [4]:
INPUT_UNITS = 28
NUM_HIDDEN_UNITS = 31
BATCH_SIZE = 128
MAX_SEQ_LEN = 28

In [5]:
train_loop_count = mnist.train.num_examples // BATCH_SIZE
test_loop_count  = mnist.test.num_examples // BATCH_SIZE

In [6]:
from minimalrnn import MinimalRNNCell
from tensorflow.python.layers.layers import conv1d,max_pooling1d,dropout,batch_normalization

class MnistRnn:
    """
    """
    def __init__(self, inputs, labels,
                 use_custom_phi=False,
                 use_batch_norm=False,
                 use_rnn_dropout=False):
        """
        """

        training    = tf.Variable(False,name='training',dtype=tf.bool)
        global_step = tf.Variable(1,name='global_step',dtype=tf.int64)
        dropout_rate = tf.Variable(0.0,name='dropout_rate',dtype=tf.float32)

        keep_prob   = tf.cond(training,lambda: 1.0 - dropout_rate,lambda: 1.0)

        step_update = tf.placeholder(tf.int64,None,name='step_update')
        dropout_update = tf.placeholder(tf.float32,None,name='dropout_update')

        def my_phi_initializer(inputs, num_outputs, **kwargs):
            
            print(('my_phi_initializer','inputs.shape',inputs.get_shape().as_list()))
            
            def my_phi(inputs):
                
                input_units = inputs.get_shape().as_list()[-1]
                layer   = tf.reshape(inputs, [-1,input_units,1])

                layer   = conv1d(layer, num_outputs//3, 5, 1, activation=tf.nn.relu)
                layer   = max_pooling1d(layer, 3, 2)
                layer   = conv1d(layer, num_outputs, 3, 1, activation=tf.nn.relu)
                layer   = conv1d(layer, num_outputs, 3, 1, activation=tf.nn.relu)
                layer   = conv1d(layer, num_outputs//2, 3, 1, activation=tf.nn.relu)
                layer   = max_pooling1d(layer, 3, 2)

                layer   = conv1d(layer, num_outputs, 1, 1, activation=tf.nn.relu)
                layer   = dropout(layer, rate=1.0-keep_prob)
                layer   = conv1d(layer, num_outputs, 1, 1, activation=tf.nn.relu)

                layer   = tf.reduce_mean(layer, axis=1, keep_dims=True)
                layer   = tf.squeeze(layer)

                print(('my_phi','output.shape',layer.get_shape().as_list()))
                
                return layer
            
            
            return my_phi


        cell = MinimalRNNCell(NUM_HIDDEN_UNITS,
                              phi_initializer=my_phi_initializer if use_custom_phi else None)

        if use_rnn_dropout:
            cell = tf.nn.rnn_cell.DropoutWrapper(
                cell,
                state_keep_prob=keep_prob,
                variational_recurrent=True,
                dtype=tf.float32
            )
        
        sequence_length = [MAX_SEQ_LEN] * BATCH_SIZE
        
        last, states = tf.nn.dynamic_rnn(
            cell,
            inputs,
            sequence_length=sequence_length,
            dtype=tf.float32)
        rnn_output = last[:,MAX_SEQ_LEN-1,:]
        outputs    = tf.layers.dense(rnn_output, 10)
        
        loss       = tf.losses.sparse_softmax_cross_entropy(
            labels,
            outputs)

        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimize   = tf.train.AdamOptimizer(learning_rate=0.001). \
                minimize(loss)

        preds      = tf.argmax(outputs, axis=1)
        errors     = tf.count_nonzero(labels - preds)
        accuracy   = 1.0 - tf.cast(errors,tf.float32) / \
                        tf.cast(tf.size(preds), tf.float32)
        
        with tf.control_dependencies([
            tf.assign(training, True),
            tf.assign(global_step, step_update),
            tf.assign(dropout_rate, dropout_update)
        ]):
            self.train_mode = tf.constant(1)
            
        with tf.control_dependencies([
            tf.assign(training, False)
        ]):
            self.test_mode  = tf.constant(1)

        self.step_update = step_update
        self.dropout_update = dropout_update
        self.inputs   = inputs
        self.labels   = labels
        self.outputs  = outputs
        self.loss     = loss
        self.optimize = optimize
        self.accuracy = accuracy

In [7]:
def train(sess, model, max_epochs, train_writer=None, test_writer=None, dropout_rate=0.5):
    """
    """

    step = sess.run(tf.train.get_global_step())

    for ep in range(max_epochs):

        for i in range(train_loop_count):

            sess.run(model.train_mode, {
                model.step_update: step,
                model.dropout_update: dropout_rate})

            offs = i * BATCH_SIZE
            batch_input = mnist.train.images[offs:offs+BATCH_SIZE,:]. \
                            reshape([BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS])
            batch_label = mnist.train.labels[offs:offs+BATCH_SIZE]
            t_start     = time.time()
            _, loss, accuracy = sess.run(
                [model.optimize, model.loss, model.accuracy],
                 feed_dict = {
                     model.inputs: batch_input,
                     model.labels: batch_label })
            t_elapsed     = time.time() - t_start
            step += 1
            
            if train_writer:
                summary = tf.Summary(
                    value = [
                        tf.Summary.Value(tag='accuracy',simple_value=accuracy),
                        tf.Summary.Value(tag='loss',simple_value=loss),
                        tf.Summary.Value(tag='elapsed',simple_value=t_elapsed),
                    ])
                train_writer.add_summary(summary, global_step=step)
                train_writer.flush()
                
            if step % 251 == 0:
                print('[train] ep {:d}, step {:d}, accu {:.5f}, loss {:.5f} [elapsed {:.5f}]'.format(
                    ep + 1, step, accuracy, loss, t_elapsed))
                
        test_accuracies = []
        
        sess.run(model.test_mode)

        for i in range(test_loop_count):
            offs = i * BATCH_SIZE
            batch_input = mnist.test.images[offs:offs+BATCH_SIZE,:]. \
                            reshape([BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS])
            batch_label = mnist.test.labels[offs:offs+BATCH_SIZE]
            t_start   = time.time()
            accuracy, = sess.run([model.accuracy],
                                 feed_dict = {
                                     model.inputs: batch_input,
                                     model.labels: batch_label})
            t_elapsed = time.time() - t_start
            test_accuracies.append(accuracy)
            
            if test_writer:
                summary = tf.Summary(
                    value = [
                        tf.Summary.Value(tag='accuracy',simple_value=accuracy),
                        tf.Summary.Value(tag='elapsed',simple_value=t_elapsed),
                    ])
                test_writer.add_summary(summary, global_step=step)
                test_writer.flush()
                
            if i % 503 == 0:
                print(' [test] ep {:d}, step {:d}, accu {:.5f} [elapsed {:.5f}]'.format(
                    ep + 1, step, np.mean(test_accuracies), t_elapsed))


In [8]:
tf.reset_default_graph()
inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')
model   = MnistRnn(inputs_, labels_)

In [9]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess   = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
train_writer = tf.summary.FileWriter('logdir/train1')
test_writer  = tf.summary.FileWriter('logdir/test1')

In [10]:
train(sess, model, 50, train_writer, test_writer)

[train] ep 1, step 251, accu 0.54688, loss 1.53519 [elapsed 0.01117]
 [test] ep 1, step 430, accu 0.68750 [elapsed 0.00944]
[train] ep 2, step 502, accu 0.69531, loss 0.94320 [elapsed 0.00977]
[train] ep 2, step 753, accu 0.79688, loss 0.63411 [elapsed 0.00990]
 [test] ep 2, step 859, accu 0.83594 [elapsed 0.00543]
[train] ep 3, step 1004, accu 0.83594, loss 0.45320 [elapsed 0.01252]
[train] ep 3, step 1255, accu 0.90625, loss 0.31008 [elapsed 0.01066]
 [test] ep 3, step 1288, accu 0.87500 [elapsed 0.00417]
[train] ep 4, step 1506, accu 0.88281, loss 0.38491 [elapsed 0.00973]
 [test] ep 4, step 1717, accu 0.89062 [elapsed 0.00448]
[train] ep 5, step 1757, accu 0.92188, loss 0.32144 [elapsed 0.00967]
[train] ep 5, step 2008, accu 0.86719, loss 0.36720 [elapsed 0.00947]
 [test] ep 5, step 2146, accu 0.92969 [elapsed 0.00542]
[train] ep 6, step 2259, accu 0.92188, loss 0.25181 [elapsed 0.00998]
[train] ep 6, step 2510, accu 0.96875, loss 0.14967 [elapsed 0.00961]
 [test] ep 6, step 2575, 

[train] ep 47, step 19829, accu 0.97656, loss 0.10964 [elapsed 0.01080]
[train] ep 47, step 20080, accu 0.96094, loss 0.17524 [elapsed 0.01129]
 [test] ep 47, step 20164, accu 0.97656 [elapsed 0.00444]
[train] ep 48, step 20331, accu 0.97656, loss 0.09736 [elapsed 0.00930]
[train] ep 48, step 20582, accu 0.96875, loss 0.08364 [elapsed 0.00932]
 [test] ep 48, step 20593, accu 0.97656 [elapsed 0.00522]
[train] ep 49, step 20833, accu 0.99219, loss 0.02994 [elapsed 0.00954]
 [test] ep 49, step 21022, accu 0.97656 [elapsed 0.00485]
[train] ep 50, step 21084, accu 0.98438, loss 0.06853 [elapsed 0.00978]
[train] ep 50, step 21335, accu 0.98438, loss 0.05367 [elapsed 0.00996]
 [test] ep 50, step 21451, accu 0.97656 [elapsed 0.00422]


In [11]:
tf.reset_default_graph()
inputs2_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels2_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')
model2   = MnistRnn(inputs2_, labels2_, use_custom_phi = True)

('my_phi_initializer', 'inputs.shape', [128, 28])
('my_phi', 'output.shape', [128, 31])


In [12]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess   = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
train_writer2 = tf.summary.FileWriter('logdir/train2')
test_writer2  = tf.summary.FileWriter('logdir/test2')

In [13]:
train(sess, model2, 50, train_writer2, test_writer2)

[train] ep 1, step 251, accu 0.54688, loss 1.52188 [elapsed 0.05732]
 [test] ep 1, step 430, accu 0.65625 [elapsed 0.02680]
[train] ep 2, step 502, accu 0.69531, loss 1.14380 [elapsed 0.04807]
[train] ep 2, step 753, accu 0.67188, loss 1.00010 [elapsed 0.04667]
 [test] ep 2, step 859, accu 0.74219 [elapsed 0.01850]
[train] ep 3, step 1004, accu 0.82031, loss 0.63754 [elapsed 0.04778]
[train] ep 3, step 1255, accu 0.85156, loss 0.56439 [elapsed 0.05032]
 [test] ep 3, step 1288, accu 0.78906 [elapsed 0.01817]
[train] ep 4, step 1506, accu 0.80469, loss 0.53878 [elapsed 0.04853]
 [test] ep 4, step 1717, accu 0.85156 [elapsed 0.01544]
[train] ep 5, step 1757, accu 0.79688, loss 0.52974 [elapsed 0.04974]
[train] ep 5, step 2008, accu 0.87500, loss 0.45741 [elapsed 0.06600]
 [test] ep 5, step 2146, accu 0.89062 [elapsed 0.01638]
[train] ep 6, step 2259, accu 0.87500, loss 0.40388 [elapsed 0.04743]
[train] ep 6, step 2510, accu 0.92969, loss 0.29526 [elapsed 0.07458]
 [test] ep 6, step 2575, 

[train] ep 47, step 19829, accu 0.97656, loss 0.09635 [elapsed 0.04918]
[train] ep 47, step 20080, accu 0.97656, loss 0.07947 [elapsed 0.05264]
 [test] ep 47, step 20164, accu 0.99219 [elapsed 0.01553]
[train] ep 48, step 20331, accu 0.97656, loss 0.07327 [elapsed 0.04830]
[train] ep 48, step 20582, accu 0.98438, loss 0.04497 [elapsed 0.04936]
 [test] ep 48, step 20593, accu 0.99219 [elapsed 0.01552]
[train] ep 49, step 20833, accu 0.98438, loss 0.08219 [elapsed 0.06899]
 [test] ep 49, step 21022, accu 0.99219 [elapsed 0.01562]
[train] ep 50, step 21084, accu 0.97656, loss 0.06770 [elapsed 0.04635]
[train] ep 50, step 21335, accu 0.98438, loss 0.05689 [elapsed 0.05019]
 [test] ep 50, step 21451, accu 0.99219 [elapsed 0.02574]


In [14]:
!tensorboard --logdir logdir

E1214 17:49:34.696614 MainThread program.py:255] TensorBoard attempted to bind to port 6006, but it was already in use
TensorBoard attempted to bind to port 6006, but it was already in use
